In [ ]:
data_root = 'data/drums/'

In [ ]:
from os.path import join
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import collections
import pickle
import sklearn
import json
import re

In [ ]:
metadata = json.load(open(join(data_root, 'metadata.json')))

In [ ]:
docs = [' '.join([m['name'], m['description']] + m['tags']) for m in metadata]
docs = [d.replace('-', '_') for d in docs] # replace dash with underscore

In [ ]:
# print the `limit` most common tokens
limit = 50
vectorizer = CountVectorizer(min_df=10, stop_words='english', binary=True)
vectors = vectorizer.fit_transform(docs)
freqs = [(vectors.getcol(idx).sum(), word) for word, idx in vectorizer.vocabulary_.items()]
for freq, token in sorted(freqs, key=lambda x: -x[0])[:limit]:
    print freq, token

`synsets.json` should be a list of synsets, where each synset is a list of synonyms. A synonym should not contain characters like spaces that a tokenizer would split into two tokens. An example:

```json
[
 ["bass"],
 ["kick", "kickdrum", "kicks"],
 ["hat", "hihat", "hi_hat"],
 ["snare", "snares"]
]
```

In [ ]:
# vectorize only using words in the synsets
synsets = json.load(open(join(data_root, 'synsets.json')))
vocabulary = [item for sublist in synsets for item in sublist]
vectorizer = CountVectorizer(min_df=1, stop_words='english', binary=True, vocabulary=vocabulary)
vectors = vectorizer.fit_transform(docs)

In [ ]:
samples_to_labels = [v.nonzero()[1] for v in vectors]
labels_to_samples = [v.nonzero()[1] for v in vectors.T]

In [ ]:
with open(join(data_root, 'labels_to_samples.pkl'), 'wb') as f:
    pickle.dump(labels_to_samples, f)
with open(join(data_root, 'samples_to_labels.pkl'), 'wb') as f:
    pickle.dump(samples_to_labels, f)